In [1]:
import torch

In [2]:
embedding_layer = torch.nn.Embedding(10, 3)

In [3]:
embedding_layer.weight

Parameter containing:
tensor([[-1.9576e-01,  9.0685e-01, -2.3725e-01],
        [-6.5495e-01, -4.4356e-01, -5.6937e-01],
        [-7.6300e-01,  7.8565e-01, -1.1423e-01],
        [ 1.6167e+00,  5.8848e-01, -9.2545e-01],
        [-4.5452e-01,  1.7633e+00, -5.0979e-01],
        [-1.7944e+00, -1.3099e+00, -2.6892e-01],
        [ 4.7739e-01,  2.1953e-01, -7.2605e-01],
        [-6.8092e-02, -5.1862e-01,  9.6706e-01],
        [-3.5239e-04,  1.7474e+00,  5.9033e-01],
        [-1.1159e-02, -1.5465e+00,  4.9035e-01]], requires_grad=True)

In [6]:
put = torch.LongTensor([[1,2,4,5],[4,3,2,9]])

In [6]:
embedding_layer(put)

tensor([[[-0.6550, -0.4436, -0.5694],
         [-0.7630,  0.7856, -0.1142],
         [-0.4545,  1.7633, -0.5098],
         [-1.7944, -1.3099, -0.2689]],

        [[-0.4545,  1.7633, -0.5098],
         [ 1.6167,  0.5885, -0.9254],
         [-0.7630,  0.7856, -0.1142],
         [-0.0112, -1.5465,  0.4903]]], grad_fn=<EmbeddingBackward>)

In [7]:
torch.full(
            (len(put),), -1, dtype=torch.long, device=torch.cuda
        )

TypeError: full() received an invalid combination of arguments - got (tuple, int, device=module, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, Number fill_value, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, Number fill_value, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


卷积神经网络 学习

In [2]:
import torch
from torch import nn

In [2]:
X = torch.tensor([[0, 1, 2], [3,4,5], [6, 7, 8]])
K = torch.tensor([[0,1],[2,3]])

In [3]:
def calcorr(X, K):
    Y = torch.zeros((X.shape[0] - K.shape[0] + 1, X.shape[1] - K.shape[1] + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+K.shape[0], j:j+K.shape[1]] * K).sum() # 无
    return Y
calcorr(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
K = torch.tensor([[1.0, -1.0]])


In [7]:
calcorr(X, K)

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [6]:
class PositionalEncoding(nn.Module):
    """位置编码"""
    def __init__(self, num_hiddens, dropout, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        # 创建一个足够长的P
        self.P = torch.zeros((1, max_len, num_hiddens))
        X = torch.arange(max_len, dtype=torch.float32).reshape(
            -1, 1) / torch.pow(10000, torch.arange(
            0, num_hiddens, 2, dtype=torch.float32) / num_hiddens)
        self.P[:, :, 0::2] = torch.sin(X)
        self.P[:, :, 1::2] = torch.cos(X)

    def forward(self, X):
        X = X + self.P[:, :X.shape[1], :].to(X.device)
        return self.dropout(X)

pos_encoding = PositionalEncoding(512, 0.1)